In [1]:
# for collab

import tensorflow as tf

def colab_connect():

    if tf.__version__!='2.13.0':
        %pip install tensorflow==2.13.0
        print('Stopping RUNTIME! Please run again.')
        import os
        os.kill(os.getpid(), 9)

    from google.colab import drive
    drive.mount('/content/drive/',force_remount=True)
    %cd /content/drive/My Drive/LOB/

    %pip install automodinit

try:
    import google.colab
    IS_COLAB = True
    colab_connect()
except:
    IS_COLAB =False


In [2]:
import numpy as np
from tqdm import tqdm
import keras

import data
from utilites import DataClass

seq_len = 100

In [3]:
# Load data
row_data = (
    # data.load_dataset(horizon=4)
    data.load_saved_datas(1000))

data.inspect_datas(row_data)

    Datas:
train     : x= (1000, 40)      | y= (1000,)        
val       : x= (1000, 40)      | y= (1000,)        
test      : x= (1000, 40)      | y= (1000,)        


In [5]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

# Test model

In [6]:
from models import m_preln as test_model

In [7]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=512,
    seq_len=seq_len,
)
(ds_train, ds_val, ds_test) =\
(datasets['train'], datasets['val'], datasets['test'])
data.inspect_datasets(datasets)

    Datasets:
train     : [2, 100, 40]
val       : [2, 100, 40]
test      : [2, 100, 40]


In [8]:
# Build
model_name = 'base_model_low'
pars = DataClass(test_model.PARAMETRS)
model = test_model.build_model(**pars.Info_expanded)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 40)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 14)           1134      
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_2 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_3 (Conv1D)           (None, 100, 14)           406       
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 14)           406       
                                                                 
 layer_normalization (Layer  (None, 100, 14)           28    

In [9]:
# Callbacks

class SaveCallback(keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        if not batch%50:
            !cp -a "/content/drive/MyDrive/LOB/Temp/callbacks" "/content/drive/MyDrive/LOB/Save"


callbacks = [
    keras.callbacks.TensorBoard(
        f"Temp/callbacks/{model_name}",
        histogram_freq=1,
        update_freq=1,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        f"Temp/callbacks/{model_name}/checkPoint",
        monitor="val_loss",
        verbose=0,
        save_best_only=False,
        save_weights_only=True,
        mode="auto",
        save_freq=50,
        options=None,
        initial_value_threshold=None,
    )
]
if IS_COLAB: callbacks +[SaveCallback()]


In [10]:
# Train
model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
    callbacks=callbacks,
)

Epoch 1/20
2/2 [==============================] - 8s 4s/step - loss: 3.4825 - sp_acc: 0.2142 - acc: 0.2042 - val_loss: 2.5622 - val_sp_acc: 0.2653 - val_acc: 0.0300
Epoch 2/20
2/2 [==============================] - 3s 2s/step - loss: 2.5888 - sp_acc: 0.3651 - acc: 0.5971 - val_loss: 2.4778 - val_sp_acc: 0.3374 - val_acc: 0.0222
Epoch 3/20
2/2 [==============================] - 3s 2s/step - loss: 2.4409 - sp_acc: 0.4340 - acc: 0.3607 - val_loss: 2.6205 - val_sp_acc: 0.3785 - val_acc: 0.0078
Epoch 4/20
2/2 [==============================] - 3s 2s/step - loss: 2.5743 - sp_acc: 0.4206 - acc: 0.2297 - val_loss: 2.5609 - val_sp_acc: 0.3796 - val_acc: 0.0155
Epoch 5/20
2/2 [==============================] - 3s 2s/step - loss: 2.5374 - sp_acc: 0.4451 - acc: 0.3019 - val_loss: 2.4238 - val_sp_acc: 0.3840 - val_acc: 0.1032
Epoch 6/20
2/2 [==============================] - 3s 2s/step - loss: 2.5412 - sp_acc: 0.4584 - acc: 0.4750 - val_loss: 2.3534 - val_sp_acc: 0.3918 - val_acc: 0.3452
Epoch 7/20

KeyboardInterrupt: 

# Search

In [ ]:
%tensorboard 

UsageError: Line magic function `%tensorboard` not found.


In [4]:
import datetime

from keras_tuner.tuners import GridSearch
import keras_tuner

import models

In [5]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=100,
    seq_len=seq_len,
)
(ds_train, ds_val, ds_test) =\
(datasets['train'], datasets['val'], datasets['test'])
data.inspect_datasets(datasets)

    Datasets:
train     : [10, 100, 40]
val       : [10, 100, 40]
test      : [10, 100, 40]


In [56]:
# Tuner parametrs
def configure(hp: keras_tuner.HyperParameters):

    class CN_search(DataClass):
        dilation_steps = hp.Int(
            'cn_layers',
            default=4,
            min_value=3,
            max_value=5,
            step=1,
        )

    class AN_search(DataClass):
        attention_heads = 3
        blocks = 2
        share_weights = hp.Boolean(
            'share_weights',
            default=True,
        )

    class FF_search(DataClass):
        dropout_rate = hp.Float(
            'dropout_rate',
            default=0.1,
            min_value=0,
            max_value=0.5,
            step=0.1,
        )

    lr = hp.Float(
        'lr',
        default=1e-4,
        min_value=1e-6,
        max_value=1e-3,
    )
    beta1 = hp.Float(
        'beta1',
        default=0.9,
        min_value=0.5,
        max_value=1.1,
        step=0.1,
    )
    beta2 = hp.Float(
        'beta1',
        default=0.999,
        min_value=0.5,
        max_value=1.1,
        sampling='log',
    )

    class Full_search(DataClass):
        cn = CN_search()
        an = AN_search()
        ff = FF_search()
        optimizer = keras.optimizers.Adam(
            name="Adam",
            learning_rate=lr,
            beta_1=beta1,
            beta_2=beta2,
        )

    return Full_search()

In [57]:
# Build model
model_name = 'search_test3'
log_dir = f'Temp/callbacks/{model_name}_{datetime.datetime.now().strftime("%H-%M-%S__%d-%m")}'
def search_base_model(hp):
    hyper_pars_data = configure(hp)
    pars_data = DataClass( models.m_base.PARAMETRS)
    pars = pars_data.Info_expanded
    pars.update(hyper_pars_data.Info_expanded)
    
    model = models.m_base.build_model(**pars)
    return model
log_dir

'Temp/callbacks/13-19-05__28-09'

In [58]:
# Build tuner
tuner = keras_tuner.RandomSearch(
    hypermodel=search_base_model,
    objective="loss",
    executions_per_trial=5,
    directory=log_dir,
    project_name=model_name,
    
)

In [59]:
# Callbacks
class SaveCallback(keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        if not batch%50:
            !cp -a "/content/drive/MyDrive/LOB/Temp/callbacks" "/content/drive/MyDrive/LOB/Save"


callbacks = [
    keras.callbacks.TensorBoard(
        log_dir= log_dir,
        histogram_freq=1,
        update_freq=1,
    ),
    # tf.keras.callbacks.ModelCheckpoint(
    #     f"{save_path}/checkPoints",
    #     monitor="val_loss",
    #     verbose=0,
    #     save_best_only=False,
    #     save_weights_only=True,
    #     mode="auto",
    #     save_freq=50,
    #     options=None,
    #     initial_value_threshold=None,
    # )
]
if IS_COLAB: callbacks +[SaveCallback()]

In [60]:
# Train
tuner.search(
    ds_train,
    validation_data=ds_val,
    epochs=5,
    callbacks=callbacks,
)


Trial 5 Complete [00h 02m 05s]
loss: 1.7429562568664552

Best loss So Far: 1.7429562568664552
Total elapsed time: 00h 09m 48s
INFO:tensorflow:Oracle triggered exit
